In [1]:
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
path_to_data = "./Data/"
import pandas as pd

In [2]:
x_train = pd.read_csv(path_to_data + 'Xtr.csv', sep = ',',header=None)
to_drop = x_train.columns[-1]
x_train = x_train.drop(to_drop,axis=1)

In [ ]:
x_train2 = genfromtxt(path_to_data + 'Xtr.csv', delimiter = ',')

In [3]:
y_train = pd.read_csv(path_to_data + 'ytr.csv', sep = ',')

In [4]:
x_test = pd.read_csv(path_to_data + 'Xte.csv', sep = ',',header=None)
to_drop = x_test.columns[-1]
x_test = x_test.drop(to_drop,axis=1)

In [ ]:
x_test2 = genfromtxt(path_to_data + 'Xte.csv', delimiter = ',')

In [5]:
import svm_solver
import kernels
import cvxopt

In [ ]:
# reshape the arrays
def ar_reshape(array, image_size = 1024):
    reshaped_ar = array.T[:-1].T.reshape(len(array),3,image_size)
    result = np.empty([len(array), image_size, 3])
    for i in range(len(reshaped_ar)):
        result[i] = reshaped_ar[i].T
    return result

In [ ]:
x_tr_rs = ar_reshape(x_train2)
x_te_rs = ar_reshape(x_test2)

In [ ]:
# show images

def im_show(array, image_size = 1024):
    size = np.sqrt(image_size)
    reshaped_ar = array.reshape(size, size, 3)
    plt.gray()
    plt.imshow(reshaped_ar)
    plt.show()


In [ ]:
for i in range(1):
    im_show(x_tr_rs[i]+0.5)
    print(y_train.Prediction[i])

# Classification : basic SVM classifier with scikit learn

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3],
                     'C': [100]}]
#tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                     'C': [1, 10, 100, 1000]},
#                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']


for score in scores:
    clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(x_train[:,:-1], y_train[:,-1])

    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    y_true, y_pred = y_test, clf.predict(x_test[:,:-1])
    print(classification_report(y_true, y_pred))
    print()

## Classification with svm solver 1 vs all :

In [6]:
clfmulti = svm_solver.SVM_multiclass(kernels.linear_kernel,C=1,nb_classes=np.unique(y_train.Prediction.values).size)


In [7]:
np.unique(y_train.Prediction.values).size

10

In [8]:
K = pd.read_csv("kernel_matrix_linear.csv",sep=',',header=None)

In [ ]:
K

In [9]:
clfmulti.fit(x_train.values,y_train.Prediction.values,precomputed=True,p_kernel=K.values)

Fitting all classifiers
Computing mu
Computing b
Computing mu
Computing b
Computing mu
Computing b
Computing mu
Computing b
Computing mu
Computing b
Computing mu
Computing b
Computing mu
Computing b
Computing mu
Computing b
Computing mu
Computing b
Computing mu
Computing b


In [16]:
np.sign(clfmulti.clf[0].predict_margin(x_train.values[0::100]))

Computing kernel matrix...
Computing decision...


array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [17]:
np.sign(clfmulti.clf[1].predict_margin(x_train.values[0::100]))==1

Computing kernel matrix...
Computing decision...


array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False], dtype=bool)

In [18]:
y_train.Prediction.values[0::100]

array([8, 7, 0, 1, 0, 6, 5, 8, 6, 2, 1, 5, 7, 0, 6, 6, 2, 1, 3, 3, 3, 2, 1,
       9, 9, 7, 8, 7, 9, 4, 7, 3, 8, 2, 5, 7, 3, 8, 2, 2, 2, 5, 8, 5, 7, 3,
       8, 0, 6, 3])

In [20]:
predictions = np.asarray([clfmulti.clf[i].predict_margin(x_train.values[0:100]) for i in range(clfmulti.nb_classes)])

Computing kernel matrix...
Computing decision...
Computing kernel matrix...
Computing decision...
Computing kernel matrix...
Computing decision...
Computing kernel matrix...
Computing decision...
Computing kernel matrix...
Computing decision...
Computing kernel matrix...
Computing decision...
Computing kernel matrix...
Computing decision...
Computing kernel matrix...
Computing decision...
Computing kernel matrix...
Computing decision...
Computing kernel matrix...
Computing decision...


In [24]:
(np.argmax(predictions,axis=0)==y_train.Prediction.values[0:100]).sum()

87